In [38]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
print(len(os.listdir("samples")))

1071


In [40]:
import string 

symbols = string.ascii_lowercase + "0123456789"

num_symbols = len(symbols)
img_shape = (50, 200 , 1)

print(num_symbols)

36


In [41]:
def preprocess_data():
    n_samples = len(os.listdir('samples/samples'))
    X = np.zeros((n_samples, 50, 200, 1)) #1071*50*200
    y = np.zeros((5, n_samples, num_symbols)) #5*1070*36
    
    for i, pic in enumerate(os.listdir('samples/samples')):
        img = cv2.imread(os.path.join('samples/samples', pic), cv2.IMREAD_GRAYSCALE)
        pic_target = pic[:-4]
        if len(pic_target) < 6:
            img = img /255.0
            img = np.reshape(img, (50, 200 ,1))
            
            targs = np.zeros((5, num_symbols))
            for j, l in enumerate(pic_target):
                ind = symbols.find(l)
                targs[j, ind] = 1
            X[i] = img
            y[:, i] = targs
            
    return X, y

X, y = preprocess_data()
X_train, y_train = X[:970], y[:, :970]
X_test, y_test = X[970:], y[:, 970:]
        

In [42]:
def create_model():
    img = layers.Input(shape = img_shape)
    
    conv1 = tf.keras.layers.Conv2D(16, (3, 3), padding = 'same', activation = 'relu')(img)
    maxpool1 = tf.keras.layers.MaxPooling2D(padding = 'same')(conv1)
    
    conv2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(maxpool1)
    maxpool2 = tf.keras.layers.MaxPooling2D(padding = 'same')(conv2)
    
    conv3 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(maxpool2)
    batch_norm = tf.keras.layers.BatchNormalization()(conv3)
    maxpool3 = tf.keras.layers.MaxPooling2D(padding = 'same')(batch_norm)
    
    flat = tf.keras.layers.Flatten()(maxpool3)
    outs = []
    for _ in range(5):
        dense1 = tf.keras.layers.Dense(64, activation = 'relu')(flat)
        dropout = tf.keras.layers.Dropout(0.5)(dense1)
        dense2 = layers.Dense(num_symbols, activation = 'sigmoid')(dropout)
        
        outs.append(dense2)
        
    model = Model(img, outs)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [43]:
model = create_model()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 50, 200, 1)] 0                                            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 50, 200, 16)  160         input_5[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_12 (MaxPooling2D) (None, 25, 100, 16)  0           conv2d_12[0][0]                  
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 25, 100, 32)  4640        max_pooling2d_12[0][0]           
____________________________________________________________________________________________

In [44]:
hist = model.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], batch_size = 32, epochs = 30, verbose = 1, validation_split = 0.2)

Epoch 1/30
25/25 [==============================] - 5s 203ms/step - loss: 17.3725 - dense_32_loss: 3.2350 - dense_34_loss: 3.5708 - dense_36_loss: 3.5861 - dense_38_loss: 3.4664 - dense_40_loss: 3.5141 - dense_32_accuracy: 0.0709 - dense_34_accuracy: 0.0451 - dense_36_accuracy: 0.0528 - dense_38_accuracy: 0.0528 - dense_40_accuracy: 0.0490 - val_loss: 17.7457 - val_dense_32_loss: 3.7634 - val_dense_34_loss: 3.5048 - val_dense_36_loss: 3.5399 - val_dense_38_loss: 3.4595 - val_dense_40_loss: 3.4781 - val_dense_32_accuracy: 0.0000e+00 - val_dense_34_accuracy: 0.0515 - val_dense_36_accuracy: 0.0979 - val_dense_38_accuracy: 0.0979 - val_dense_40_accuracy: 0.0515
Epoch 2/30
25/25 [==============================] - 4s 172ms/step - loss: 16.1761 - dense_32_loss: 2.9310 - dense_34_loss: 3.3346 - dense_36_loss: 3.3673 - dense_38_loss: 3.2729 - dense_40_loss: 3.2704 - dense_32_accuracy: 0.0838 - dense_34_accuracy: 0.0412 - dense_36_accuracy: 0.0606 - dense_38_accuracy: 0.0696 - dense_40_accuracy:

25/25 [==============================] - 4s 177ms/step - loss: 7.8634 - dense_32_loss: 0.6226 - dense_34_loss: 1.3074 - dense_36_loss: 1.9216 - dense_38_loss: 2.2024 - dense_40_loss: 1.8094 - dense_32_accuracy: 0.8131 - dense_34_accuracy: 0.5593 - dense_36_accuracy: 0.3441 - dense_38_accuracy: 0.1675 - dense_40_accuracy: 0.3892 - val_loss: 13.6034 - val_dense_32_loss: 4.4116 - val_dense_34_loss: 1.9686 - val_dense_36_loss: 2.4152 - val_dense_38_loss: 2.5059 - val_dense_40_loss: 2.3020 - val_dense_32_accuracy: 0.0619 - val_dense_34_accuracy: 0.6959 - val_dense_36_accuracy: 0.4794 - val_dense_38_accuracy: 0.1804 - val_dense_40_accuracy: 0.6031
Epoch 14/30
25/25 [==============================] - 4s 180ms/step - loss: 6.9520 - dense_32_loss: 0.5017 - dense_34_loss: 1.0758 - dense_36_loss: 1.6573 - dense_38_loss: 2.0801 - dense_40_loss: 1.6372 - dense_32_accuracy: 0.8325 - dense_34_accuracy: 0.6186 - dense_36_accuracy: 0.4510 - dense_38_accuracy: 0.1959 - dense_40_accuracy: 0.4562 - val_lo

25/25 [==============================] - 4s 178ms/step - loss: 2.1737 - dense_32_loss: 0.1819 - dense_34_loss: 0.2991 - dense_36_loss: 0.4977 - dense_38_loss: 0.6231 - dense_40_loss: 0.5719 - dense_32_accuracy: 0.9291 - dense_34_accuracy: 0.8827 - dense_36_accuracy: 0.8209 - dense_38_accuracy: 0.7590 - dense_40_accuracy: 0.7887 - val_loss: 12.7092 - val_dense_32_loss: 10.0924 - val_dense_34_loss: 0.6629 - val_dense_36_loss: 0.8282 - val_dense_38_loss: 0.6953 - val_dense_40_loss: 0.4304 - val_dense_32_accuracy: 0.1031 - val_dense_34_accuracy: 0.8608 - val_dense_36_accuracy: 0.7732 - val_dense_38_accuracy: 0.8093 - val_dense_40_accuracy: 0.8351
Epoch 26/30
25/25 [==============================] - 4s 180ms/step - loss: 2.0759 - dense_32_loss: 0.1665 - dense_34_loss: 0.2465 - dense_36_loss: 0.5066 - dense_38_loss: 0.5986 - dense_40_loss: 0.5576 - dense_32_accuracy: 0.9472 - dense_34_accuracy: 0.9137 - dense_36_accuracy: 0.8170 - dense_38_accuracy: 0.7693 - dense_40_accuracy: 0.7938 - val_l

In [45]:
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = img / 255.0
    else:
        print("Image not Detected")
    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (5, 36))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        
    capt = ''
    for l in l_ind:
        capt +=symbols[l]
        
    return capt


In [46]:
score = model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]], verbose = 1)
print('Test Loss and accuracy: ', score)

4/4 [==============================] - 0s 28ms/step - loss: 17.3721 - dense_32_loss: 15.7406 - dense_34_loss: 0.1977 - dense_36_loss: 0.6201 - dense_38_loss: 0.5851 - dense_40_loss: 0.2286 - dense_32_accuracy: 0.0000e+00 - dense_34_accuracy: 0.9100 - dense_36_accuracy: 0.8600 - dense_38_accuracy: 0.8200 - dense_40_accuracy: 0.9300
Test Loss and accuracy:  [17.372095108032227, 15.740581512451172, 0.19774363934993744, 0.6200811862945557, 0.5850576162338257, 0.22863321006298065, 0.0, 0.9100000262260437, 0.8600000143051147, 0.8199999928474426, 0.9300000071525574]


In [47]:
model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])
print(predict('samples/samples/8n5p3.png'))
print(predict('samples/samples/f2m8n.png'))
print(predict('samples/samples/dce8y.png'))
print(predict('samples/samples/3eny7.png'))
print(predict('samples/samples/npxb7.png'))

4/4 [==============================] - 0s 26ms/step - loss: 17.3721 - dense_32_loss: 15.7406 - dense_34_loss: 0.1977 - dense_36_loss: 0.6201 - dense_38_loss: 0.5851 - dense_40_loss: 0.2286 - dense_32_accuracy: 0.0000e+00 - dense_34_accuracy: 0.9100 - dense_36_accuracy: 0.8600 - dense_38_accuracy: 0.8200 - dense_40_accuracy: 0.9300
8n5p3
f2m8n
dce8y
3eny7
mpxf7
